In [ ]:
"""
Pregătire mediu de lucru
Stabilirea conexiunii dintre Google drive si Colab notebook
"""
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Instalarea PySpark

!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://dlcdn.apache.org/spark/
!wget -q https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
!tar xf spark-3.4.4-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"

import findspark
findspark.init()
findspark.find()
import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
  .builder \
  .getOrCreate()

spark

In [ ]:
data_type ='json'
path      = '/content/drive/MyDrive/Colab Notebooks/Data/Data/practice/'+ data_type
data_df   = spark.read.format(data_type).load(path)
data_df.show(10, truncate = False)

In [ ]:
data_df.printSchema()

In [ ]:
data_df.show()

In [ ]:
data_type     = 'parquet'
path          = '/content/drive/MyDrive/Colab Notebooks/Data/Data/practice/'+ data_type
data_practice = spark.read.format(data_type).load(path)
data_practice.show()

In [ ]:
from pyspark.sql import functions as f

In [ ]:
data_practice.count()

In [ ]:
data_practice.show(truncate=False)

Curățarea coloanei „nume” de spații suplimentare - Se recomandă folosirea funcției trim sau replace.

Curățarea coloanei „ocupatie” de sparții suplimentare -  Se recomandă folosirea funcției trim sau replace.

Eliminarea diacriticelor din coloana „nume” -

Scoaterea valorilor duplicate din coloana „extra”

Setarea coloanei „inactiv” pe false atunci când valoarea lipsește. - Se recomandă folosirea funcției coalesce.


In [ ]:
from pyspark.sql.functions import col, lit, trim, coalesce, translate, array_distinct, concat, when, size, array_join, desc_nulls_last, desc

In [ ]:
df_tasks = (
    data_practice
    .withColumn('nume', trim(col('nume'))   )
    .withColumn('ocupatie',trim(col('ocupatie')))
    .withColumn('inactiv',coalesce(col('inactiv'),f.lit(False) ) )
    .withColumn('nume',translate(col('nume'),'șțăîâ','staia' ))
    .withColumn('extra',array_distinct(col('extra')))
)

In [ ]:
df_tasks.show()

Adăugarea coloanei „varstă_la_contractare” egală cu diferența dintre vârsta persoanei și vechime

Scoaterea coloanei „vechime”

Adăugarea coloanei „text_descriptiv” care să fie un text ce descrie clientul și conține neapărat toate informațiile știute despre el, mai puțin despre starea contractului dacă este activ. (e.g. X, în vârstă de ... ani,
este ...). Atenție la cazul special la coloana „extra” când numărul de elemente este 0. -  Se recomandă cel folosirea funcțiilor when și printf.

In [ ]:
df_transformare = (
    data_practice
    .withColumn('varsta_la_contractare',col('varsta') - col('vechime'))
    .drop(col('vechime'))
  #  .withColumn('array_extra', array_join(col('extra'),', '))
  #  .withColumn('extra cnt', size(col('extra') ) )
    .withColumn('text_descriptiv',
                trim(
                    concat(
                          col('nume'),
                          lit(', in varsta de '),
                          col('varsta'),
                          lit(' ani, este '),
                          col('ocupatie'),

                          when(
                              size(f.col('extra') ) > 0 ,
                                concat(
                                      lit(' cu extra activitati in : ') ,
                                      array_join(col('extra'),', '))
                          )
                          .otherwise(lit(''))

                      )                                                                              ))
)

In [ ]:
df_transformare.explain('extended')

In [ ]:
df_transformare.show(truncate=False)

### Afișarea clienților inactivi / Afișarea contractelor inactive

In [ ]:
df_inactive_contracts = df_tasks.filter('inactiv is True')
df_inactive_contracts.show(20)
#df_inactive_contracts.count()


### Afișarea clienților cu minim și maxim de vechime / Afișarea contractelor care au minim 2 ani de vechime și maxim 5 ani vechime, inclusiv.

In [ ]:
df_vechime_2y_to_5y = (
    df_tasks
    .filter(col('vechime') >= 2)
    .filter(col('vechime') <= 5)
)
df_vechime_2y_to_5y.show(20)

### Afișarea celor mai în vârstă persoane cu orice fel de contract  / Afișarea top 10 contracte sortate după vârsta lor descrescător

In [ ]:
df_invarsta = (
    df_tasks
    .sort(desc_nulls_last(col('varsta')))
    .limit(10)
    )

df_invarsta.show()

### Afișarea celor mai în vârstă oameni la contractare / Afișarea top 5 contracte active sortate după vârsta la contractare descrescător

In [ ]:
df_4 = (
    df_tasks
    .withColumn('varsta la contractare',(col('varsta') - col('vechime')))
)
df_4.sort(desc('varsta la contractare')).limit(5).show()

Reutilizarea Operațiilor

 Împărțiți setul de date în patru segmente, folosindu-vă de coloana "inactiv" și criteriul vârstei.

 Creați un segment pentru cei cu vârsta de până la 30 de ani inclusiv și un altul pentru cei peste 30 de ani, aplicând
această divizare atât pentru înregistrările active, cât și pentru cele inactive.

 Scrieți fiecare set de date într-o locație diferită.

 Înainte de scrierea fiecărui segment, afișați datele (nu uitați să cache-uiți setul de date și înainte de
afișare, nu numai înainte de împărțire)

In [ ]:

cached_data_df = df_tasks.select('varsta', 'inactiv').cache()

df_active_young = (
    cached_data_df
    .filter(col('varsta') <=  lit(30) )
     .filter('inactiv is false')
    .cache()
)


df_inactive_young = (
    cached_data_df
    .filter(col('varsta') <=  lit(30) )
    .filter('inactiv is True')
    .cache()
)

df_active_old = (
    cached_data_df
    .filter(col('varsta') >  lit(30) )
    .filter('inactiv is False')
    .cache()
)

df_inactive_old = (
    cached_data_df
    .filter(col('varsta') >  lit(30) )
    .filter('inactiv is True')
    .cache()
)




In [ ]:
df_active_young.show(10)

In [ ]:
df_inactive_young.show(10)

In [ ]:
df_active_old.show(10)

In [ ]:
df_inactive_old.show()

In [ ]:

data_format   = 'parquet'
output_folder = '/content/drive/MyDrive/Colab Notebooks/Data/Data/output/parquet/'
df_active_young.write.format(data_format).save(output_folder + '1') # datele df_active_young se vor salva in folderul 1
df_inactive_young.write.format(data_format).save(output_folder + '2') # datele df_inactive_young se vor salva in folderul 2
df_active_old.write.format(data_format).save(output_folder + '3')
df_inactive_old.write.format(data_format).save(output_folder + '4')


In [ ]:
data_without_personal_columns_df = data_practice.drop('varsta', 'extra')

In [ ]:
data_without_personal_columns_df.show(10)

In [ ]:
df_rename = data_without_personal_columns_df.withColumnRenamed('ocupatie','job')

In [ ]:
df_rename.show(10)

In [ ]:
df_rename.explain('extended')